In [45]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import pandas as pd # to read csv and handle dataframe

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable


import torchvision
import torchvision.transforms as transforms

matplotlib.rcParams['figure.figsize'] = (20,10)

data = np.load('../data/cullpdb+profile_5926.npy')
data = data.reshape((-1,700,57))

data.shape
x = data[:,:,:22]
y = data[:,:,22:33] # (brug 35 hvis du vil have solvent properties med)
print(x.shape)
print(y.shape)

X_train = x[:5430]
y_train = y[:5430]

X_test = x[5435:5690]
y_test = y[5435:5690]
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(5926, 700, 22)
(5926, 700, 11)
(5430, 700, 22)
(5430, 700, 11)
(255, 700, 22)
(255, 700, 11)


In [41]:
BATCH_SIZE = 32

torch_X_train = torch.from_numpy(X_train).type(torch.LongTensor)
torch_y_train = torch.from_numpy(y_train).type(torch.LongTensor) # data type is long

# create feature and targets tensor for test set.
torch_X_test = torch.from_numpy(X_test).type(torch.LongTensor)
torch_y_test = torch.from_numpy(y_test).type(torch.LongTensor) # data type is long

In [48]:
print(torch.cuda.is_available())

False


In [35]:
torch_X_train = torch_X_train.view(-1, 22, 700).float()
torch_X_test = torch_X_test.view(-1, 22, 700).float()
print(torch_X_train.shape)
print(torch_X_test.shape)

# Pytorch train and test sets
train = torch.utils.data.TensorDataset(torch_X_train,torch_y_train)
test = torch.utils.data.TensorDataset(torch_X_test,torch_y_test)

# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = BATCH_SIZE, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = BATCH_SIZE, shuffle = False)

torch.Size([5430, 22, 700])
torch.Size([255, 22, 700])


In [36]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(22, 32, kernel_size=5)
        self.conv2 = nn.Conv1d(32, 32, kernel_size=5)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(3*3*64, 256)
        self.fc2 = nn.Linear(256, 11)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool1d(self.conv2(x), 2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(F.max_pool1d(self.conv3(x),2))
        x = F.dropout(x, p=0.5, training=self.training)
        x = x.view(-1,3*3*64 )
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
 
cnn = CNN()
print(cnn)

it = iter(train_loader)
X_batch, y_batch = next(it)
print(X_batch.shape)
print(cnn.forward(X_batch).shape)

CNN(
  (conv1): Conv1d(22, 32, kernel_size=(5,), stride=(1,))
  (conv2): Conv1d(32, 32, kernel_size=(5,), stride=(1,))
  (conv3): Conv1d(32, 64, kernel_size=(5,), stride=(1,))
  (fc1): Linear(in_features=576, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=11, bias=True)
)
torch.Size([32, 22, 700])
torch.Size([608, 11])


In [19]:
def fit(model, train_loader):
    optimizer = torch.optim.Adam(model.parameters())#,lr=0.001, betas=(0.9,0.999))
    error = nn.CrossEntropyLoss()
    EPOCHS = 5
    model.train()
    for epoch in range(EPOCHS):
        correct = 0
        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            var_X_batch = Variable(X_batch).float()
            var_y_batch = Variable(y_batch)
            optimizer.zero_grad()
            output = model(var_X_batch)
            loss = error(output, var_y_batch)
            loss.backward()
            optimizer.step()

            # Total correct predictions
            predicted = torch.max(output.data, 1)[1] 
            correct += (predicted == var_y_batch).sum()
            #print(correct)
            if batch_idx % 50 == 0:
                print('Epoch : {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Accuracy:{:.3f}%'.format(
                    epoch, batch_idx*len(X_batch), len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.data[0], float(correct*100) / float(BATCH_SIZE*(batch_idx+1))))

In [21]:
fit(cnn, train_loader)

ValueError: Expected input batch_size (608) to match target batch_size (32).